#### Imports

In [1]:
import pandas as pd
import sqlite3 as sq

#### Import from Old SQLite Store

In [3]:
conn = sq.connect("oldstore.db")

In [3]:
book = pd.read_sql_query(f"SELECT * from Book", conn)

In [4]:
book.head()

,id,pub_id,title,price,category,quantity,b_format,prod_year,filesize
0,0399149392,21,Chesapeake Blue,25.95,Literature & Fiction,41,epub,2002,7676
1,156158648X,61,The New City Home: Smart Solutions for Metro L...,24.95,Home Design,30,leaflet,2003,1042
2,0156006219,10,The Magician's Assistant,13.00,Romance,33,epub,1998,5220
3,0440237556,41,The Dark Highlander,6.99,Romance,69,pdf,2002,55
4,038572179X,54,Atonement,14.00,Literature & Fiction,77,leaflet,2003,5817


#### Author Table ✅

In [5]:
author = pd.read_sql_query(f"SELECT * from Source", conn)

In [6]:
# Cast Table Column to String
author['s_name'] = author['s_name'].astype('string')

# Rename Columns
author.rename(columns={'s_name': 'name'}, inplace=True)

In [7]:
author.head(8)

,id,name
0,1,Ian H. Witten
1,2,Ken Henderson
2,3,Sue Monk Kidd
3,4,Chuck Palahniuk
4,5,Larry Bossidy
5,6,Ralph Kimball
6,7,Scott Kelby
7,8,Mark Z. Danielewski


#### Book Author Table ✅

In [8]:
bookauthor = pd.read_sql_query(f"SELECT * from SourceProduct", sq.connect("oldstore.db"))

In [9]:
# Rename Columns
bookauthor.rename(columns={'s_id': 'author_id', 'prod_id': 'book_id'}, inplace=True)

#Clean Book_ID Column
bookauthor['book_id'] = bookauthor['book_id'].str.strip('X')
bookauthor['book_id'] = bookauthor['book_id'].str.replace(('-'),"")

In [10]:
bookauthor.head(8)

,id,author_id,book_id
0,1,25,0743467523
1,2,96,0440237556
2,3,31,0385494246
3,4,54,156158648
4,5,22,0061020656
5,6,43,0671004107
6,7,25,0743457358
7,8,22,0061092177


#### Category Table ✅

In [11]:
category = pd.DataFrame({'category': book["category"].unique()})

# Fix ID table with offset index 
category.index = category.index + 1
category['id'] = category.index

# Fix Category Column Types
category['category'] = category['category'].astype('string')

# Reorder, Rename, and Reset Index 
category = category[['id', 'category']]
category.rename(columns={'category': 'name'},inplace=True)
category.reset_index(inplace=True, drop=True)

In [12]:
category

,id,name
0,1,Literature & Fiction
1,2,Home Design
2,3,Romance
3,4,Science
4,5,Fantasy
5,6,Computer
6,7,Accounting & Finance
7,8,Horror
8,9,Reference
9,10,Fiction


#### Start Book Table | Clean 
Drop Columns to Match our new SQLite DB Columns

In [13]:
book.columns

Index(['id', 'pub_id', 'title', 'price', 'category', 'quantity', 'b_format',
       'prod_year', 'filesize'],
      dtype='object')

In [14]:
book = book.drop(columns=['pub_id', 'filesize'])

Clean ID Column

In [15]:
book['id'] = book['id'].str.strip('X')
book['id'] = book['id'].str.replace(('-'),"")

Rename and Reorder Columns

In [16]:
book.rename(columns={'prod_year': 'year', 'category': 'category_id'}, inplace=True)
book['rating'] = 100
book = book[['id', 'title', 'price', 'year', 'quantity', 'rating', 'category_id']]

#### Book Table | Merge

In [17]:
book = pd.merge(book, category, how='left', left_on='category_id', right_on='name')
book['category_id'] = book['id_y']

book.drop(columns=['id_y', 'name'], inplace=True)
book.rename(columns={'id_x': 'id'}, inplace=True)

#### Book Table ✅

In [18]:
book.head(8)

,id,title,price,year,quantity,rating,category_id
0,0399149392,Chesapeake Blue,25.95,2002,41,100,1
1,156158648,The New City Home: Smart Solutions for Metro L...,24.95,2003,30,100,2
2,0156006219,The Magician's Assistant,13.00,1998,33,100,3
3,0440237556,The Dark Highlander,6.99,2002,69,100,3
4,038572179,Atonement,14.00,2003,77,100,1
5,1590520734,The Rescuer: The O'Malley Series,12.99,2003,64,100,3
6,0373218257,Love By Design,14.95,2003,94,100,1
7,0060958405,Words and Rules: The Ingredients of Language,15.00,2000,77,100,4


In [19]:
conn.close()

#### Export **book**, **author**, **bookauthor**, and **category** to our new SQLlite Store Dataset

In [5]:
conn2 = conn = sq.connect("bookstore.db")

In [21]:
book.to_sql(name='book', con=conn2, if_exists='append', index=False)

109

In [22]:
author.to_sql(name='author', con=conn2, if_exists='append', index=False)

102

In [23]:
bookauthor.to_sql(name='bookauthor', con=conn2, if_exists='append', index=False)

141

In [24]:
category.to_sql(name='category', con=conn2, if_exists='append', index=False)

10

In [6]:
conn2.close()